In [2]:
from dolfinx import mesh, fem, io
import ufl
from mpi4py import MPI
from petsc4py.PETSc import ScalarType
import numpy as np
import matplotlib.pyplot as plt

from dolfinx.fem import Constant, FunctionSpace,Function, dirichletbc,locate_dofs_geometrical
from ufl import dx,ds,grad,TrialFunction,TestFunction, FacetNormal, lhs,rhs, form


class Infix:

    def __init__(self, function):
        self.function = function

    def __ror__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))

    def __or__(self, other):
        return self.function(other)

    def __call__(self, value1, value2):
        return self.function(value1, value2)


def all_boundary(domain, space):
    domain.topology.create_connectivity(
        domain.topology.dim - 1,
        domain.topology.dim,
        )
    boundary_facets = mesh.exterior_facet_indices(domain.topology)
    boundary_dofs = fem.locate_dofs_topological(
        space,
        domain.topology.dim - 1,
        boundary_facets,
        )
    return boundary_dofs


dot = Infix(ufl.inner)


def L_errors(uS, def_uEx):
    domain = uS.function_space.mesh

    V_err = FunctionSpace(
        domain, ('CG', uS.function_space.element.space_dimension)
        )
    uEx = Function(V_err)
    uEx.interpolate(def_uEx)
    L1_norm = form((uS-uEx) * dx)
    L2_norm = form((uS - uEx)**2 * dx)
    L1_scalar = fem.assemble_scalar(L1_norm)
    L2_scalar = fem.assemble_scalar(L2_norm)

    L1_err = np.abs(domain.comm.allreduce(L1_scalar, op=MPI.SUM))
    L2_err = np.sqrt(domain.comm.allreduce(L2_scalar, op=MPI.SUM))
    return (L1_err, L2_err)


def graph2D(fig, lists):

    def data_construct(geometry, x_array):
        data = np.column_stack((geometry[:, 0:2], x_array))
        x_data = data[:, 0]
        y_data = data[:, 1]
        z_data = data[:, 2]
        return [x_data, y_data, z_data]

    for list in lists:
        u, axes, title = list

        domain = u.function_space.mesh
        geometry = domain.geometry.x

        levels = np.linspace(u.x.array.min(), u.x.array.max(), 10)
        axes.set_title(title)
        plot = axes.tricontourf(
            *data_construct(geometry, u.x.array), levels=levels
            )
        axes.set_aspect(1)
        fig.colorbar(plot, ax=axes)
    return fig